# Dummy Variables Exercise

In this exercise, you'll create dummy variables from the projects data set. The idea is to transform categorical data like this:

| Project ID | Project Category |
|------------|------------------|
| 0          | Energy           |
| 1          | Transportation   |
| 2          | Health           |
| 3          | Employment       |

into new features that look like this:

| Project ID | Energy | Transportation | Health | Employment |
|------------|--------|----------------|--------|------------|
| 0          | 1      | 0              | 0      | 0          |
| 1          | 0      | 1              | 0      | 0          |
| 2          | 0      | 0              | 1      | 0          |
| 3          | 0      | 0              | 0      | 1          |


(Note if you were going to use this data with a model influenced by multicollinearity, you would want to eliminate one of the columns to avoid redundant information.) 

The reasoning behind these transformations is that machine learning algorithms read in numbers not text. Text needs to be converted into numbers. You could assign a number to each category like 1, 2, 3, and 4. But a categorical variable has no inherent order, so you want to reflect this in your features.

Pandas makes it very easy to create dummy variables with the [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) method. In this exercise, you'll create dummy variables from the World Bank projects data; however, there's a caveat. The World Bank data is not particularly clean, so you'll need to explore and wrangle the data first.

You'll focus on the text values in the sector variables.

Run the code cells below to read in the World Bank projects data set and then to filter out the data for text variables. 

In [1]:
import pandas as pd
import numpy as np

# read in the projects data set and do basic wrangling 
projects = pd.read_csv('../data/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# keep the project name, lending, sector and theme data
sector = projects.copy()
sector = sector[['project_name', 'lendinginstr', 'sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector',
          'mjsector1', 'mjsector2', 'mjsector3', 'mjsector4', 'mjsector5',
          'mjsector', 'theme1', 'theme2', 'theme3', 'theme4', 'theme5', 'theme ',
          'goal', 'financier', 'mjtheme1name', 'mjtheme2name', 'mjtheme3name',
          'mjtheme4name', 'mjtheme5name']]

Run the code cell below. This cell shows the percentage of each variable that is null. Notice the mjsector1 through mjsector5 variables are all null. The mjtheme1name through mjtheme5name are also all null as well as the theme variable. 

Because these variables contain so many null values, they're probably not very useful.

In [3]:
# output percentage of values that are missing
100 * sector.isnull().mean()

project_name      0.000000
lendinginstr      1.348093
sector1           0.000000
sector2          47.791539
sector3          64.450899
sector4          76.019290
sector5          85.132617
sector            3.496274
mjsector1       100.000000
mjsector2       100.000000
mjsector3       100.000000
mjsector4       100.000000
mjsector5       100.000000
mjsector          3.496274
theme1            0.000000
theme2           46.005042
theme3           58.987286
theme4           71.317405
theme5           83.954406
theme           100.000000
goal             33.510522
financier        61.310829
mjtheme1name    100.000000
mjtheme2name    100.000000
mjtheme3name    100.000000
mjtheme4name    100.000000
mjtheme5name    100.000000
dtype: float64

## Exercise 1

The sector1 variable looks promising; it doesn't contain any null values at all. In the next cell, store the unique sector1 values in a list and output the results. Use the sort_values() and unique() methods.

In [4]:
# TODO: Create a list of the unique values in sector1. Use the sort_values() and unique() pandas methods. 
# And then convert those results into a Python list
uniquesectors1 = sector['sector1'].sort_values().unique()
uniquesectors1

array(['!$!0', '(Historic)Agency reform!$!50!$!AG',
       '(Historic)Agricultural credit!$!100!$!AC', ...,
       'Workforce Development and Vocational Education!$!96!$!EW',
       'Workforce Development and Vocational Education!$!98!$!EW',
       'Workforce Development and Vocational Education!$!99!$!EW'],
      dtype=object)

In [5]:
# run this code cell to see the number of unique values
print('Number of unique values in sector1:', len(uniquesectors1))

Number of unique values in sector1: 3060


3060 different categories is quite a lot! Remember that with dummy variables, if you have n categorical values, you need n - 1 new variables! That means 3059 extra columns!

## Exercise 2

There are a few issues with this 'sector1' variable. First, there are values labeled '!$!0'. These should be substituted with NaN.

Furthermore, each sector1 value ends with a ten or eleven character string like '!$!49!$!EP'. Some sectors show up twice in the list like:
 'Other Industry; Trade and Services!$!70!$!YZ',
 'Other Industry; Trade and Services!$!63!$!YZ',

But it seems like those are actually the same sector. You'll need to remove everything past the exclamation point. 

Many values in the sector1 variable start with the term '(Historic)'. Try removing that phrase as well.

### replace() method

With pandas, you can use the replace() method to search for text and replace parts of a string with another string. If you know the exact string you're looking for, the replace() method is straight forward. For example, say you wanted to remove the string '(Trial)' from this data:

| data                     |
|--------------------------|
| '(Trial) Banking'        |
| 'Banking'                |
| 'Farming'                |
| '(Trial) Transportation' |

You could use `df['data'].replace('(Trial)', '')` to replace (Trial) with an empty string.

### regular expressions
What about this data?

| data                                           |
|------------------------------------------------|
| 'Other Industry; Trade and Services?$ab' |
| 'Other Industry; Trade and Services?ceg' |

This type of data is trickier. In this case, there's a pattern where you want to remove a string that starts with an exclamation point and then has an unknown number of characters after it. When you need to match patterns of character, you can use [regular expressions](https://en.wikipedia.org/wiki/Regular_expression).

The replace method can take a regular expression. So
df['data'].replace('?.+', regex=True) where '?.+' means find a set of characters that starts with a question mark is then followed by one or more characters. You can see a [regular expression cheat sheet](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285) here.

Fix these issues in the code cell below.

In [32]:
# TODO: In the sector1 variable, replace the string '!$!0' with nan
# HINT: you can use the pandas replace() method and numpy.nan
sector['sector1'] = sector['sector1'].replace('!$!0', np.nan)
sector['sector1'] = sector['sector1'].replace('\(\D.+\)','',regex=True) 
sector['sector1'] = sector['sector1'].replace('!.+','',regex=True)

# TODO: In the sector1 variable, remove the last 10 or 11 characters from the sector1 variable.
# HINT: There is more than one way to do this including the replace method
# HINT: You can use a regex expression '!.+'
# That regex expression looks for a string with an exclamation
# point followed by one or more characters

# TODO: Remove the string '(Historic)' from the sector1 variable
# HINT: You can use the replace method

print('Number of unique sectors after cleaning:', len(list(sector['sector1'].unique())))
print('Percentage of null values after cleaning:', 100 * sector['sector1'].isnull().sum() / sector['sector1'].shape[0])

Number of unique sectors after cleaning: 156
Percentage of null values after cleaning: 3.4962735642262164


Now there are 156 unique categorical values. That's better than 3060. If you were going to use this data with a supervised learning machine model, you could try converting these 156 values to dummy variables. You'd still have to train and test a model to see if those are good features.

You could try to consolidate similar categories together, which is what the challenge exercise in part 4 is about.

There are also still many entries with NaN values. How could you fill these in?

You might try to determine an appropriate category from the 'project_name' or 'lendinginstr' variables. If you make dummy variables including NaN values, then you could consider a feature with all zeros to represent NaN. Or you could delete these records from the data set. Pandas will ignore NaN values by default. That means, for a given row, all dummy variables will have a value of 0 if the sector1 value was NaN.

Don't forget about the bigger context! This data is being prepared for a machine learning algorithm. Whatever techniques you use to engineer new features, you'll need to use those when running your model on new data. So if your new data does not contain a sector1 value, you'll have to run whatever feature engineering processes you did on your training set.

## Exercise 3

In this next exercise, use the pandas pd.get_dummies() method to create dummy variables. Then use the concat() method to concatenate the dummy variables to a dataframe that contains the project totalamt variable and the project year from the boardapprovaldate.

In [39]:
projects['boardapprovaldate'].dt.year

0        2018.0
1        2018.0
2        2018.0
3        2018.0
4        2018.0
          ...  
18243    1948.0
18244    1947.0
18245    1947.0
18246    1947.0
18247    1947.0
Name: boardapprovaldate, Length: 18248, dtype: float64

In [40]:
# TODO: Create dummy variables from the sector1 data. Put the results into a dataframe called dummies
# Hint: Use the get_dummies method
dummies = pd.get_dummies(sector['sector1'])

# TODO: Create a new dataframe called df by 
#       filtering the projects data for the totalamt and
#       the year from boardapprovaldate
projects['year'] = projects['boardapprovaldate'].dt.year
df = projects[['year','totalamt']]

# TODO: Concatenate the results of dummies and projects
#       into a single data frame
df_final = pd.concat([dummies,df],axis=1)

df_final.head()

,Adult; Basic and Continuing Education,Agency reform,Agricultural Extension; Research; and Other Support Activities,Agricultural credit,Agricultural extension,Agricultural markets; commercialization and agri-business,Agriculture adjustment,Agro-industry,Agro-industry and marketing,Animal production,...,Urban management,Urban transport,Urban water supply,Vocational training,Waste Management,Water Supply,Water supply and sanitation adjustment,Workforce Development and Vocational Education,year,totalamt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018.0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018.0,200000000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018.0,58330000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018.0,20000000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018.0,100000000


You could continue to consolidate sector values using other techniques. For example, in the next exercise, you'll find categories with similar terms and then combine them together. 

Keep in mind that how much to consolidate will depend on your machine learning model performance and your hardware's ability to handle the extra features in memory. If your hardware's memory can handle 3060 new features and your machine learning algorithm performs better, then go for it!

## Exercise 4 (Challenge)

But can you do anything else with the sector1 variable?

The percentage of null values for 'sector1' is now 3.49%. That turns out to be the same number as the null values for the 'sector' column. You can see this if you scroll back up to where the code calculated the percentage of null values for each variable. 

Perhaps the 'sector1' and 'sector' variable have the same information. If you look at the 'sector' variable, however, it also needs cleaning. The values look like this:

'Urban Transport;Urban Transport;Public Administration - Transportation'

It turns out the 'sector' variable combines information from the 'sector1' through 'sector5' variables and the 'mjsector' variable. Run the code cell below to look at the sector variable.

In [41]:
sector['sector']

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                                                      NaN
4        Social Protection;Social Protection;Other Indu...
                               ...                        
18243                      (Historic)Hydro;(Historic)Hydro
18244    (Historic)Economic management;(Historic)Econom...
18245    (Historic)Economic management;(Historic)Econom...
18246    (Historic)Economic management;(Historic)Econom...
18247    (Historic)Economic management;(Historic)Econom...
Name: sector, Length: 18248, dtype: object

In [52]:
sector['sector'] = sector['sector'].replace('\(\D.+\)','',regex=True).replace(';.+','',regex=True)

In [54]:
sector['sector'].unique()

array([nan, 'Social Protection', 'Urban Transport',
       'Other Public Administration',
       'Workforce Development and Vocational Education',
       'Energy Transmission and Distribution', 'Renewable Energy Solar',
       'Other Industry', 'Health', 'Water Supply', 'Primary Education',
       'Other Water Supply',
       'Public Administration - Energy and Extractives',
       'Central Government ', 'ICT Services',
       'Rural and Inter-Urban Roads', 'Public Administration - Education',
       'Sub-National Government', 'Other Education',
       'Other Transportation', 'Renewable Energy Hydro',
       'Public Administration - Agriculture',
       'Public Administration - Transportation', 'Oil and Gas',
       'Other Non-bank Financial Institutions', 'Ports/Waterways',
       'Public Administration - Financial Sector', 'Mining',
       'Public Administration - Social Protection',
       'Agricultural Extension', 'Tertiary Education',
       'Secondary Education', 'Crops', 'Housin

In [73]:
sector['sector_agg'] = sector['sector'].replace(np.nan,'').apply(lambda s: 'Agriculture' if 'Agricul' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Energy' if 'Energy' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Economy' if 'Econom' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Social' if 'Social' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Water' if 'Water' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Finace' if 'Finan' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Education' if 'Educ' in s else s)
sector['sector_agg'] = sector['sector_agg'].apply(lambda s: 'Transport' if 'Transp' in s else s)

In [75]:
sector['sector_agg'].unique().shape

(121,)

# Conclusion

Pandas makes it relatively easy to create dummy variables; however, oftentimes you'll need to clean the data first.